# Chat History

An example of a client interacting with a chatbot where message history is persisted on the backend.

In [15]:
import uuid
from langserve import RemoteRunnable

chat = RemoteRunnable("http://localhost:8000/test/")

Let's create a prompt composed of a system message and a human message.

In [16]:
session_id = str(uuid.uuid4())

In [18]:
chat.invoke(
    {"input": "How much cow with id 101 produced in Jun 2020"},
    config={"configurable": {"session_id": session_id}},
)

HTTPStatusError: Client error '400 Bad Request' for url 'http://localhost:8000/test/invoke'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400 for {"detail":"Session ID `` is not in a valid format. Session ID must only contain alphanumeric characters, hyphens, and underscores."}

In [4]:
chat.invoke({"human_input": "what was my name?"}, {'configurable': { 'session_id': session_id } })

AIMessage(content=' You told me your name is Eugene.')

In [5]:
chat.invoke({"human_input": "What animal do i like?"}, {'configurable': { 'session_id': session_id } })

AIMessage(content=' You said you like cats.')

In [7]:
for chunk in chat.stream({'human_input': "Can you count till 10?"},  {'configurable': { 'session_id': session_id } }):
    print()
    print(chunk.content, end='', flush=True)


 Sure
,
 I
'd
 be
 happy
 to
 count
 to
 10
:


1
,
 2
,
 3
,
 4
,
 5
,
 6
,
 7
,
 8
,
 9
,
 10


In [12]:
!cat chat_histories/c7a327f3-5578-4fb7-a8f2-3082d7cb58cc.json | jq .

[
  {
    "type": "human",
    "data": {
      "content": "my name is eugene. i like cats. what is your name?",
      "additional_kwargs": {},
      "type": "human",
      "example": false
    }
  },
  {
    "type": "ai",
    "data": {
      "content": " Hello Eugene! My name is Claude. It's nice to meet another cat lover.",
      "additional_kwargs": {},
      "type": "ai",
      "example": false
    }
  },
  {
    "type": "human",
    "data": {
      "content": "what was my name?",
      "additional_kwargs": {},
      "type": "human",
      "example": false
    }
  },
  {
    "type": "ai",
    "data": {
      "content": " You told me your name is Eugene.",
      "additional_kwargs": {},
      "type": "ai",
      "example": false
    }
  },
  {
    "type": "human",
    "data": {
      "content": "What animal do i like?",
      "additional_kwargs": {},
      "type": "human",
      "example": false
    }
  },
  {
    "type": "ai",
    "data": {
      "content": " You said you like cats.

In [ ]:

def get_db():
    return SQLDatabase.from_databricks(
        catalog = "hive_metastore", 
        schema = "default",
        host = os.getenv("HOST"),
        api_token = os.getenv("API_TOKEN_DATABRICKS"),
        cluster_id = os.getenv("CLUSTER_ID")
    )

In [2]:


from langchain_community.chat_message_histories import FileChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_community.llms import Ollama
from langchain.memory import ChatMessageHistory
from langchain.agents import AgentType
# from langchain_ollama.llms import OllamaLLM

from langchain_core.messages import HumanMessage
from langchain_ollama.chat_models import ChatOllama
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.agent_toolkits import create_sql_agent

from ChatBot_V1.db.db import get_db

# llm = ChatOllama(model="llama3") 
# toolkit = SQLDatabaseToolkit(db=get_db(), llm=llm) 
# agent_executor = create_sql_agent(llm, toolkit=toolkit, agent_type="tool-calling")

# agent = create_sql_agent(llm=llm,db=get_db(),verbose=True,agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

/home/dev-002/Desktop/Implementation of chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm2 = ChatOllama(model="gemma2")

In [5]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain

write_query = create_sql_query_chain(llm, get_db())
execute_query = QuerySQLDataBaseTool(db=get_db())

In [6]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)



'I see what\'s going on here!\n\nThe SQL query provided is not actually a valid SQL query, but rather a text explanation of the situation. There is no actual SQL query to run or analyze.\n\nIn other words, there is no answer to the question "How many employees are there" because it\'s based on a non-existent piece of information (i.e., "employees" table or column).\n\nIf you\'d like to ask a different question that can be answered by the given data, I\'d be happy to help!'

In [9]:
chain.invoke({"question": "Give me age of cow with id -2147483212"})

KeyboardInterrupt: 

In [10]:

# memory = ChatMessageHistory(session_id="test-session")

# agent_with_chat_history = RunnableWithMessageHistory(
#     agent,
#     lambda session_id: memory,
#     input_messages_key="input",
#     history_messages_key="chat_history",
#     )
    
# response=agent_with_chat_history.invoke(
#     {"input": "Give me age of cow with id -2147483212"},
#     config={"configurable": {"session_id": "test-session"}},
# )

agent.invoke("Give me age of cow with id -2147483212")



> Entering new SQL Agent Executor chain...
Let's start by listing the tables in the database.

Action: sql_db_list_tables
Action Input: (empty stringdataproviderswithoutlinkedstreamer, dp, filtereddf, healtheventsexploded, healtheventsnonexploded, healtheventsnonexplodedmultifile, herd_data_provider_info, lifespan_prediction, marjan_delta_streamer_health_team, milkproduction, milkrecordingsexploded, milkrecordingsnonexploded, milkrecordingsnonexplodedmultifile, miso_delta_streamer_health, miso_delta_streamer_health_team, predicted_diagnoses_merged, ratings, receiveddatadetailsperdataprovider, sh, streamerhealth_overview, streamerhealthoverviewdelta, streamerherdoverview, streamerherdoverviewnewdeltanew, streamertimingsperdataproviderLet's take a closer look at the tables. Since we're dealing with cows and their ages, I'm going to focus on the "milkproduction" and "lifespan_prediction" tables.

Action: sql_db_schema
Action Input: milkproduction, lifespan_prediction
CREATE TABLE lifesp

KeyboardInterrupt: 

In [5]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm2, db=get_db(), verbose=True)

In [6]:
agent_executor.invoke(
    {
        "Give me age of cow with id -2147483212"
    }
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: dataproviderswithoutlinkedstreamer, dp, filtereddf, healtheventsexploded, healtheventsnonexploded, healtheventsnonexplodedmultifile, herd_data_provider_info, lifespan_prediction, marjan_delta_streamer_health_team, milkproduction, milkrecordingsexploded, milkrecordingsnonexploded, milkrecordingsnonexplodedmultifile, miso_delta_streamer_health, miso_delta_streamer_health_team, predicted_diagnoses_merged, ratings, receiveddatadetailsperdataprovider, sh, streamerhealth_overview, streamerhealthoverviewdelta, streamerherdoverview, streamerherdoverviewnewdeltanew, streamertimingsperdataproviderThought:  The tables that might have information about cows are herd_data_provider_info, milkproduction, milkrecordingsexploded, milkrecordingsnonexploded, milkrecordingsnonexplodedmultifile. I should query the schema of these tables.
Action: sql_db_schema 
Action Input: herd_data_provider_info, milkproduction, milkrec

KeyboardInterrupt: 

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.agent_toolkits import create_sql_agent

llm = ChatOllama(model="llama3-groq-tool-use:latest", base_url="{url}") 
toolkit = SQLDatabaseToolkit(db=db, llm=llm) 
agent_executor = create_sql_agent(llm, toolkit=toolkit, agent_type="tool-calling")
agent_executor.invoke([HumanMessage(content="A question about the database?")])